In [4]:
# Set the process name to be human readable in htop
import setproctitle
setproctitle.setproctitle("05_Fetch_Population_Variants")

from config import *

import requests
import pandas as pd
pd.options.display.max_columns = 999

import numpy as np

from tqdm import tqdm, tqdm_notebook
from tqdm._tqdm_notebook import tqdm_notebook

tqdm.pandas(tqdm_notebook)
tqdm_notebook.pandas()

from collections import defaultdict

This notebook fetches all known population variants for the human interactors included in Proteins.txt using the GnomAD API. Then (requiring some manual input) these variants are mapped through the Variant Effect Predictor (VEP) to get corresponding UniProt positions / other information.

**NOTE:** When returning to this step to reformat the code for this repository I encountered a lot of errors because some of the external API's and tools that I used seem to have changed since I originally ran the code. I've reproduced the original workflow as best as possible, but still encounter some troubles I did not originally encounter with genes failing the GnomAD API query. The example output from this notebook included in this repository is just a sub-set of my original population variants list for the human genes included in the demo interacitons list. Results could vary if you try to reproduce this from scratch.

- Inputs:
  - Proteins.txt


- Outputs:
  - VEP_Input.txt
  - Human_Population_Variants_VEP_Mapped.txt
  - Pop_Vars.txt


- Dependencies:
  - Must be run after 03_Generate_Proteins_Summary

# Query GnomAD for Population Variants

In [2]:
# Borrowed From https://gist.github.com/ressy/6fd7f6ee6401ac8e703dc2709399869e
# See also Docs at (https://gnomad.broadinstitute.org/api)
def fetch_gnomAD(jsondata, url="https://gnomad.broadinstitute.org/api"):
    # The server gives a generic error message if the content type isn't
    # explicitly set
    headers = {"Content-Type": "application/json"}
    response = requests.post(url, json=jsondata, headers=headers)
    json = response.json()
    if "errors" in json:
        raise Exception(str(json["errors"]))
    return json
# FUNCTION END

# Borrowed From https://gist.github.com/ressy/6fd7f6ee6401ac8e703dc2709399869e
# See also Docs at (https://gnomad.broadinstitute.org/api)
def queryGnomadByGene(gene_name, dataset="gnomad_r2_1", fields=["gene_id", "gene_symbol", "chrom", "pos", "ref", "alt", "consequence", "rsid", "variantId", "hgvsp"]):
    # Note that this is GraphQL, not JSON.
    fmt_graphql = """
    {
        gene(gene_symbol: "%s", reference_genome:GRCh38) {
          variants(dataset: %s) {
          %s
        }
      }
    }
    """
    # This part will be JSON encoded, but with the GraphQL part left as a
    # glob of text.
    req_variantlist = {
        "query": fmt_graphql % (gene_name, dataset, "\n".join(fields)),
        "variables": {}
        }
    response = fetch_gnomAD(req_variantlist)
    return pd.DataFrame(response["data"]["gene"]["variants"])[fields]
# FUNCTION END

In [5]:
# Read in all Human Proteins Involved in Interactome
all_proteins = pd.read_csv("{0}/Proteins.txt".format(output_dir), sep="\t")
human_genes = all_proteins[all_proteins["Is_Viral"] == False]["Gene_Name"].to_list()

In [7]:
# Fetch All Variants Listed in gnomAD (filter to only missense variants)
# NOTE: There is a query rate limit exceeded error that gets thrown if you submit
#       too many requests at once. I've jerry rigged this to just keep trying
#       until it works. But the first time I ran it, this worked fine on one attempt.
#
# NOTE: AATF and CISD3 never finish with the current setup (they throw a separate error, no data
#       in gnomad_r2_1?). These weren't a problem when I ran using GRCh37 as the reference genome
#       instead of GRCh38. Comparing the two outputs, the chromosomal positions don't seem to actually
#       change depending on the reference genome selected so I'm not sure what this means. When
#       this code was run originally, the GnomAD API didn't require a reference_genome selection.

import time
all_variants = dict()

attempt_i = 1
attempts_per_gene = 10
while(True):
    cur_keys = len(all_variants)
    if(cur_keys == len(human_genes)):
        break
    print "Starting Attempt", attempt_i, "with", cur_keys, "Genes Parsed"
    
    # Iterate over all genes
    for g in tqdm_notebook(human_genes):
        if(g in all_variants):
            continue
        
        # Try to fetch variants on each gene n times
        tries = 0
        while(True):
            try:
                tries += 1
                pop_variants = queryGnomadByGene(g)
                break
            except Exception:
                time.sleep(0.1)
                if(tries > attempts_per_gene):
                    break
                pass
        if(tries > attempts_per_gene):
            print "Skipped", g
            continue
        if(not g == pop_variants["gene_symbol"].unique()[0]):
            print "MISMATCH: ", g, "-->", pop_variants["gene_symbol"].unique()[0]
            pop_variants["gene_symbol"] = g
        all_variants[g] = pop_variants[pop_variants["consequence"] == "missense_variant"]
    
    print "Finished Attempt", attempt_i, "with", len(all_variants), "Genes Parsed (", (len(all_variants) - cur_keys), " New)"
    print
    attempt_i += 1
    time.sleep(60)
#all_variants = pd.concat(all_variants)

Starting Attempt 1 with 0 Genes Parsed


Skipped ARF6
Skipped DDX21
Skipped PIGO
Skipped SLC27A2
Skipped SLC44A2

Finished Attempt 1 with 5 Genes Parsed ( 5  New)

Starting Attempt 2 with 5 Genes Parsed


Skipped ARF6
Skipped DDX21

Finished Attempt 2 with 8 Genes Parsed ( 3  New)

Starting Attempt 3 with 8 Genes Parsed


Skipped ARF6
Skipped DDX21

Finished Attempt 3 with 8 Genes Parsed ( 0  New)



KeyboardInterrupt: 

In [8]:
# Combine Variants for each gene into one DF
all_variants = pd.concat(all_variants.values())

In [9]:
# Sort and remove duplicates (no longer necessary?)
# (for some reason POL1A had duplicate entries?)
# (Could not reproduce, but GnomAD API syntax has also changed since first run)
print len(all_variants[all_variants.duplicated(["chrom", "pos", "ref", "alt"], keep=False)])

all_variants["chrom"] = all_variants["chrom"].map(lambda x: int(x) if not x == "X" else x)
print len(all_variants)
all_variants = all_variants.drop_duplicates().sort_values(["chrom", "pos", "ref", "alt"])
print len(all_variants)

0
2720
2720


# Map Variants to UniProt Position using VEP

In [76]:
# NOTE: This step can most easilly be done by manually querying VEP

In [11]:
# Write Input File for VEP
out = open("{0}/VEP_Input.txt".format(output_dir), "w+")
out.write("\n".join(all_variants[["chrom", "pos", "ref", "alt"]].apply(lambda x: "{0} {1} . {2} {3}".format(*x), axis=1)))
out.close()

In [12]:
# NOTE: Either I'm crazy or there's currently something up with VEP
#       When submitting to the grch37 version the SWISSPROT column
#       in the output only shows gene names (e.g. PARP_HUMAN) so the
#       text download cannnot be used to map UniProt IDs (this is part
#       of the sanity check I use to make sure the mapping is verifiable)
#
#       Instead of just submitting to the grch37 like I did originally.
#
#       (https://grch37.ensembl.org/Homo_sapiens/Tools/VEP)
#
#       I instead now need to re-map the GnomAD vairants (only provided in
#       grch37 coordinates) to grch38 using the ensembl assembly converter
#
#       (https://grch37.ensembl.org/Homo_sapiens/Tools/AssemblyConverter?db=core)
#
#       Then submit the mapped input to the regular grch38 VEP
#
#       (https://useast.ensembl.org/Homo_sapiens/Tools/VEP?)
#
#       This then becomes a pain because merging the VEP output (GRCh38) with
#       the original GnomAD (GRCh37) input is not straightforward. The Ensembl
#       assembly converrter drops some variants during the conversion without
#       indicating which ones are dropped so a 1-1 mapping based on order
#       is not possible.
#       
#       Rather than updating and re-runing this script I'm just defaulting to the
#       output from the previously working version of this code / external resources.

# Submit input file here...
#
# https://grch37.ensembl.org/Homo_sapiens/Tools/VEP
#
# Additional Configurations
#
# - Identifiers
#   + Gene Symbol
#   + Transcript version
#   + UniProt (not selected by default)
#
# - Variants and Frequency Data
#   + Find co-located known variants (Yes)
#   + Frequency Data for co-located variants
#     - 1000 Genomes global MAF
#     - gnomAD exomes (not selected by default)
#   + PubMedIDs for citations (Yes)
#
# - Additional Annotations
#   + Default options
#
# - Predictions
#   + Default options (SIFT + PolyPhen prediction and score)
#
# Filtering Options
#   + Defualt
#
# Advanced Options
#   + Default

In [20]:
# Read in VEP Mapped Variants
# Download output from VEP job
# NOTE: From results papge apply filter "Uploaded variant is missense_variant"
#       to limit to only relevant entries
#
# Save in root project directory and rename to match filename below...
vep_mapped = pd.read_csv("{0}/Human_Population_Variants_VEP_Mapped.txt".format(output_dir), sep="\t")

In [21]:
# Join on Key
vep_mapped["Key"] = vep_mapped[["Location", "Allele"]].apply(lambda x: x[0].split("-")[0] + x[1], axis=1)
all_variants["Key"] = all_variants[["chrom", "pos", "alt"]].apply(lambda x: str(x[0]) + ":" + str(x[1]) + x[2], axis=1)

# Make sure the same variants are included in both
# NOTE: This should not match anymore because of GRCh37 to GRCh38 mapping step
s1 = set(all_variants["Key"])
s2 = set(vep_mapped["Key"])
print s1 == s2

# Merge
merged = all_variants.join(vep_mapped.set_index("Key"), how="inner", on="Key")

False


In [23]:
# Get expected UniProt from original input / compare against VEP Mapping
proteins = pd.read_csv("{0}/Proteins.txt".format(output_dir), sep="\t")
gene2uniprot = proteins.set_index("Gene_Name")["ID"].to_dict()
merged["Expected_UniProt"] = merged["gene_symbol"].map(lambda x: gene2uniprot[x])

# Drop any cases with wrong UniProt mapping (there are only 2 here so its not a big deal)
print len(merged)
print len(merged[merged["SWISSPROT"] == merged["Expected_UniProt"]])
merged = merged[merged["SWISSPROT"] == merged["Expected_UniProt"]]

7616
7616


In [24]:
# Select / Rename Columns
merged["AA_Ref"] = merged["Amino_acids"].map(lambda x: x.split("/")[0])
merged["AA_Alt"] = merged["Amino_acids"].map(lambda x: x.split("/")[1])
merged["SIFT_Category"] = merged["SIFT"].map(lambda x: x.split("(")[0] if not x == "-" else np.nan)
merged["SIFT_Score"] = merged["SIFT"].map(lambda x: float(x.split("(")[1].strip(")")) if not x == "-" else np.nan)
merged["PolyPhen_Category"] = merged["PolyPhen"].map(lambda x: x.split("(")[0] if not x == "-" else np.nan)
merged["PolyPhen_Score"] = merged["PolyPhen"].map(lambda x: float(x.split("(")[1].strip(")")) if not x == "-" else np.nan)
merged = merged[["gene_symbol", "gene_id", "SWISSPROT", "chrom", "pos", "ref", "alt", "consequence", "rsid", "IMPACT", "Protein_position", "AA_Ref", "AA_Alt", "SIFT_Category", "SIFT_Score", "PolyPhen_Category", "PolyPhen_Score", "gnomAD_AF", "CLIN_SIG", "SOMATIC", "PHENO"]]

In [25]:
# Rename Columns
merged.columns = ["Gene_Symbol", "Gene_ID", "UniProt", "Chrom", "Pos", "Ref", "Alt", "Consequence", "rsID", "Imact", "AA_Pos", "AA_Ref", "AA_Alt", "SIFT_Category", "SIFT_Score", "PolyPhen_Category", "PolyPhen_Score", "gnomAD_AF", "Clinical_Significance", "Somatic", "Pheno"]

In [26]:
# Update gnomAD_AF to remove "-" blanks
merged["gnomAD_AF"] = merged["gnomAD_AF"].map(lambda x: float(x) if not x == "-" else np.nan)

In [28]:
# Make sure all mapped uniprot positions references match
protein_summary = pd.read_csv("{0}/Proteins.txt".format(output_dir), sep="\t")
uni2seq = protein_summary.set_index("ID")["Sequence"].to_dict()
merged["Accurate_Pos"] = merged[["UniProt", "AA_Pos", "AA_Ref"]].apply(lambda x: uni2seq[x[0]][x[1] - 1] == x[2] if x[1] <= len(uni2seq[x[0]]) else False, axis=1)

tmp = merged.sort_values(["Chrom", "Pos", "Accurate_Pos"], ascending=[True, True, False]).drop_duplicates(["Chrom", "Pos", "Ref", "Alt"])

In [29]:
tmp["Accurate_Pos"].mean()

0.9867042707493956

In [30]:
len(tmp)

2482

In [31]:
len(all_variants)

2720

In [32]:
# Only retain variants where the reference matches at the Uniprot position
tmp = tmp[tmp["Accurate_Pos"]]
tmp = tmp.drop("Accurate_Pos", axis=1)

In [135]:
# Save final list of population variants
tmp.to_csv("{0}/Pop_Vars.txt".format(output_dir), sep="\t", index=None)